# RNN Model

### Import Datasets

In [26]:
import pandas as pd

# Import Datasets
df1 = pd.read_csv(r'C:\Users\megan\Desktop\ML Project\processed_df1.csv')
df2 = pd.read_csv(r'C:\Users\megan\Desktop\ML Project\processed_df2.csv')
df3 = pd.read_csv(r'C:\Users\megan\Desktop\ML Project\processed_df3.csv')

### Imports

In [27]:
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import re

In [28]:
# Hyperparameters
learning_rate = 0.001
batch_size = 32
embed_dim = 100
hidden_dim = 128
num_layers = 2
dropout_rate = 0.2
epochs = 5

# Tokenizer function
def tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

# Encode labels
label_encoder = LabelEncoder()
df1['encoded_label'] = label_encoder.fit_transform(df1['sentiment'])
df3['encoded_label'] = label_encoder.transform(df3['sentiment'])

# Vocabulary building
counter = Counter()
for text in df1['cleaned_reviewText']:
    counter.update(tokenizer(text))

counter.update(['<pad>', '<unk>'])  # Add special tokens
vocab = {word: idx for idx, (word, _) in enumerate(counter.items())}

def encode(tokens):
    unk_index = vocab['<unk>']
    return [vocab[token] if token in vocab else unk_index for token in tokens]

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.texts[idx])
        indices = encode(tokens)
        return torch.tensor(indices), self.labels[idx]

# Padding
def collate_fn(batch):
    texts, labels = zip(*batch)
    padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>']).long()
    return padded, torch.tensor(labels, dtype=torch.long)

# DataLoaders
train_dataset = SentimentDataset(df1['cleaned_reviewText'].tolist(), df1['encoded_label'].tolist(), vocab, tokenizer)
test_dataset = SentimentDataset(df3['review_description'].tolist(), df3['encoded_label'].tolist(), vocab, tokenizer)

train_loader_rnn = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader_rnn = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

### RNN Model

In [ ]:
# Define RNN model
class modelRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers, dropout_rate):
        super(modelRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab['<pad>'])
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, nonlinearity='tanh')
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        output, hn = self.rnn(x)
        out = self.dropout(hn[-1])  # Take last hidden state
        return self.fc(out)

# Instantiate RNN model
model_rnn = modelRNN(vocab_size=len(vocab), embed_dim=embed_dim, hidden_dim=hidden_dim, 
                     num_classes=3, num_layers=num_layers, dropout_rate=dropout_rate)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model_rnn.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader_rnn:
        optimizer.zero_grad()
        outputs = model_rnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    avg_loss = running_loss / len(train_loader_rnn)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {round(avg_loss, 4)}, Accuracy: {round(accuracy, 4)}")

# Evaluation on  df3 (test set)
model_rnn.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader_rnn:
        outputs = model_rnn(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
accuracy = accuracy_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
print("")
print("Evaluation using Test Data(df3): ")
print(f"Accuracy: {round(accuracy, 4)}")
print(f"Recall: {round(recall, 4)}")
print(f"F1 Score: {round(f1, 4)}")

Epoch 1/5, Loss: 1.0176, Accuracy: 0.4977
Epoch 2/5, Loss: 1.0114, Accuracy: 0.5003
Epoch 3/5, Loss: 1.0355, Accuracy: 0.4728
Epoch 4/5, Loss: 1.0223, Accuracy: 0.4877
Epoch 5/5, Loss: 1.0221, Accuracy: 0.4949

Evaluation using Test Data(df3): 
Accuracy: 0.987
Recall: 0.987
F1 Score: 0.9805


### Checking Sentiment Count Values 

In [30]:
df1['sentiment'].value_counts(normalize=True)
# df3['sentiment'].value_counts(normalize=True)

sentiment
positive    0.499917
negative    0.333389
neutral     0.166694
Name: proportion, dtype: float64

In [31]:
df3['sentiment'].value_counts(normalize=True)

sentiment
positive    0.986957
neutral     0.008696
negative    0.004348
Name: proportion, dtype: float64

## Experimentation

### Evaluating using Training Set

In [32]:
# Evaluating on  df1 (training set)
model_rnn.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in train_loader_rnn:  
        outputs = model_rnn(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
accuracy = accuracy_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"[df1 Eval] Accuracy: {round(accuracy, 4)}")
print(f"[df1 Eval] Recall: {round(recall, 4)}")
print(f"[df1 Eval] F1 Score: {round(f1, 4)}")

[df1 Eval] Accuracy: 0.4999
[df1 Eval] Recall: 0.4999
[df1 Eval] F1 Score: 0.3332


### Hyperparameter Tuning

In [ ]:
# New Hyperparameters
learning_rate = 0.005   # Increase lr from .001 to .005
batch_size = 16         # Smaller batch size
embed_dim = 100
hidden_dim = 64         # Decrease from 128 to 64
num_layers = 2
dropout_rate = 0.3      # Increase  dropout_rate
epochs = 10             # Increase epochs

# Tokenizer function
def tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

# Encode labels
label_encoder = LabelEncoder()
df1['encoded_label'] = label_encoder.fit_transform(df1['sentiment'])
df3['encoded_label'] = label_encoder.transform(df3['sentiment'])

# Vocabulary building
counter = Counter()
for text in df1['cleaned_reviewText']:
    counter.update(tokenizer(text))

counter.update(['<pad>', '<unk>'])  # Add special tokens
vocab = {word: idx for idx, (word, _) in enumerate(counter.items())}

def encode(tokens):
    unk_index = vocab['<unk>']
    return [vocab[token] if token in vocab else unk_index for token in tokens]

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.texts[idx])
        indices = encode(tokens)
        return torch.tensor(indices), self.labels[idx]

# Padding
def collate_fn(batch):
    texts, labels = zip(*batch)
    padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>']).long()
    return padded, torch.tensor(labels, dtype=torch.long)

# DataLoaders
train_dataset = SentimentDataset(df1['cleaned_reviewText'].tolist(), df1['encoded_label'].tolist(), vocab, tokenizer)
test_dataset = SentimentDataset(df3['review_description'].tolist(), df3['encoded_label'].tolist(), vocab, tokenizer)

train_loader_rnn = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader_rnn = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define RNN model
class modelRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers, dropout_rate):
        super(modelRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab['<pad>'])
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, nonlinearity='tanh')
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        output, hn = self.rnn(x)
        out = self.dropout(hn[-1])  # Take last hidden state
        return self.fc(out)

# Instantiate RNN model
model_rnn = modelRNN(vocab_size=len(vocab), embed_dim=embed_dim, hidden_dim=hidden_dim, 
                     num_classes=3, num_layers=num_layers, dropout_rate=dropout_rate)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model_rnn.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, labels in train_loader_rnn:
        optimizer.zero_grad()
        outputs = model_rnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    avg_loss = running_loss / len(train_loader_rnn)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {round(avg_loss, 4)}, Accuracy: {round(accuracy, 4)}")

# Evaluation on  df3 (test set)
model_rnn.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader_rnn:
        outputs = model_rnn(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
accuracy = accuracy_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
print("")
print(f"[New HP] Accuracy: {round(accuracy, 4)}")
print(f"[New HP] Recall: {round(recall, 4)}")
print(f"[New HP] F1 Score: {round(f1, 4)}")

Epoch 1/10, Loss: 1.0438, Accuracy: 0.4644
Epoch 2/10, Loss: 1.0421, Accuracy: 0.4696
Epoch 3/10, Loss: 1.0434, Accuracy: 0.4667
Epoch 4/10, Loss: 1.0451, Accuracy: 0.47
Epoch 5/10, Loss: 1.0433, Accuracy: 0.4708
Epoch 6/10, Loss: 1.0426, Accuracy: 0.4653
Epoch 7/10, Loss: 1.0389, Accuracy: 0.4719
Epoch 8/10, Loss: 1.0436, Accuracy: 0.4681
Epoch 9/10, Loss: 1.0439, Accuracy: 0.473
Epoch 10/10, Loss: 1.0452, Accuracy: 0.4733

[New HP] Accuracy: 0.788
[New HP] Recall: 0.788
[New HP] F1 Score: 0.8703


### Dataset sizes

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load your dataset - Using df1
df_ds = pd.read_csv(r'C:\Users\megan\Desktop\ML Project\processed_df1.csv')

# Split data into training and testing
train_data, test_data = train_test_split(df_ds, test_size=0.2, random_state=42)

# Tokenization and padding function
def preprocess_data(data, tokenizer=None, max_words=10000, max_len=100):
    if tokenizer is None:
        tokenizer = Tokenizer(num_words=max_words)
        tokenizer.fit_on_texts(data['cleaned_reviewText'])
    sequences = tokenizer.texts_to_sequences(data['cleaned_reviewText'])
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer

# Convert labels to integers (sentiment)
def encode_labels(labels):
    label_encoder = LabelEncoder()
    return label_encoder.fit_transform(labels)

# Build RNN model
def ds_rnn_model(input_length, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=100, input_length=input_length))
    model.add(SimpleRNN(128, activation='relu', return_sequences=False))  # RNN layer
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Softmax for multi-class classification
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate model
def train_and_evaluate_rnn(train_data, test_data, subset_size):
    # Subset the training data
    train_subset = train_data.sample(frac=subset_size, random_state=42)
    
    # Preprocess the data
    X_train, tokenizer = preprocess_data(train_subset, max_words=10000)
    X_test, _ = preprocess_data(test_data, tokenizer=tokenizer, max_words=10000)
    
    y_train = encode_labels(train_subset['sentiment'])
    y_test = encode_labels(test_data['sentiment'])
    
    # Build and train RNN model
    model = ds_rnn_model(input_length=X_train.shape[1], num_classes=len(set(y_train)))
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
    
    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Dataset Size: {subset_size * 100}% | Accuracy: {round(accuracy, 4)}\n")

# Experiment with 25% and 50% dataset sizes
dataset_sizes = [0.25, 0.5]  # 25% and 50% dataset sizes
for size in dataset_sizes:
    train_and_evaluate_rnn(train_data, test_data, size)

Epoch 1/5
75/75 [==============================] - 7s 79ms/step - loss: 1.0222 - accuracy: 0.4842 - val_loss: 0.9892 - val_accuracy: 0.4979
Epoch 2/5
75/75 [==============================] - 6s 75ms/step - loss: 0.8415 - accuracy: 0.6137 - val_loss: 0.9004 - val_accuracy: 0.5967
Epoch 3/5
75/75 [==============================] - 6s 76ms/step - loss: 0.5441 - accuracy: 0.7692 - val_loss: 0.9337 - val_accuracy: 0.5946
Epoch 4/5
75/75 [==============================] - 5s 73ms/step - loss: 0.4205 - accuracy: 0.8446 - val_loss: 1.0510 - val_accuracy: 0.6375
Epoch 5/5
75/75 [==============================] - 1s 13ms/step - loss: 1.3067 - accuracy: 0.5983
Dataset Size: 25.0% | Accuracy: 0.5983

Epoch 1/5
150/150 [==============================] - 14s 92ms/step - loss: 0.9672 - accuracy: 0.5372 - val_loss: 0.8567 - val_accuracy: 0.6283
Epoch 2/5
150/150 [==============================] - 13s 87ms/step - loss: 0.7154 - accuracy: 0.7070 - val_loss: 0.8128 - val_accuracy: 0.6592
Epoch 3/5
150/15

In [ ]:
# Load your dataset - Using df3
df_ds = pd.read_csv(r'C:\Users\megan\Desktop\ML Project\processed_df3.csv')

train_data, test_data = train_test_split(df_ds, test_size=0.2, random_state=42)

# Tokenization and padding function
def preprocess_data(data, tokenizer=None, max_words=10000, max_len=100):
    if tokenizer is None:
        tokenizer = Tokenizer(num_words=max_words)
        tokenizer.fit_on_texts(data['review_description'])
    sequences = tokenizer.texts_to_sequences(data['review_description'])
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer

# Convert labels to integers (sentiment)
def encode_labels(labels):
    label_encoder = LabelEncoder()
    return label_encoder.fit_transform(labels)

# Build RNN model
def ds_rnn_model(input_length, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=100, input_length=input_length))
    model.add(SimpleRNN(128, activation='relu', return_sequences=False))  # RNN layer
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Softmax for multi-class classification
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate model
def train_and_evaluate_rnn(train_data, test_data, subset_size):
    # Subset the training data
    train_subset = train_data.sample(frac=subset_size, random_state=42)
    
    # Preprocess the data
    X_train, tokenizer = preprocess_data(train_subset, max_words=10000)
    X_test, _ = preprocess_data(test_data, tokenizer=tokenizer, max_words=10000)
    
    y_train = encode_labels(train_subset['sentiment'])
    y_test = encode_labels(test_data['sentiment'])
    
    # Build and train RNN model
    model = ds_rnn_model(input_length=X_train.shape[1], num_classes=len(set(y_train)))
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))
    
    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Dataset Size: {subset_size * 100}% | Accuracy: {round(accuracy, 4)}\n")

# Experiment with 25% and 50% dataset sizes
dataset_sizes = [0.25, 0.5]  # 25% and 50% dataset sizes
for size in dataset_sizes:
    train_and_evaluate_rnn(train_data, test_data, size)

Epoch 1/5
6/6 [==============================] - 1s 119ms/step - loss: 0.5903 - accuracy: 0.9728 - val_loss: 0.1009 - val_accuracy: 0.9891
Epoch 2/5
6/6 [==============================] - 1s 90ms/step - loss: 2928.1108 - accuracy: 0.9946 - val_loss: 27.3504 - val_accuracy: 0.9891
Epoch 3/5
6/6 [==============================] - 1s 96ms/step - loss: 0.0193 - accuracy: 0.9946 - val_loss: 0.1325 - val_accuracy: 0.9891
Epoch 4/5
6/6 [==============================] - 1s 90ms/step - loss: 0.1795 - accuracy: 0.9946 - val_loss: 0.2383 - val_accuracy: 0.9891
Epoch 5/5
6/6 [==============================] - 0s 10ms/step - loss: 0.1388 - accuracy: 0.9891
Dataset Size: 25.0% | Accuracy: 0.9891

Epoch 1/5
12/12 [==============================] - 2s 99ms/step - loss: 0.6604 - accuracy: 0.9375 - val_loss: 5.2902 - val_accuracy: 0.0000e+00
Epoch 2/5
12/12 [==============================] - 1s 90ms/step - loss: 0.1042 - accuracy: 0.9918 - val_loss: 6.3010 - val_accuracy: 0.0000e+00
Epoch 3/5
12/12 [==